# Agenda: Grouping and pivot tables

1. What is grouping?
2. Simple grouping
3. Choosing aggregate functions
4. Group on more than one column
5. Retrieving more than one column
6. Multiple aggregations
7. Grouping on dates and times
8. Grouper
9. Pivot tables

# What is grouping?

If we have a numeric column, we can run an aggregation (e.g., `mean` or `sum` or `count`) on it.  

If we have two columns, one numeric and one categorical, then we can ask to run an aggregation function on all of the columns for a particular value of the categorical column. 

Pandas provides us with an easy to to perform the aggregation function for each subset of the table, once for each value of the categorical.

In [1]:
import pandas as pd


In [2]:
filename = 'taxi.csv'

df = pd.read_csv(filename)

In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,40.754093,2,17.0,0.0,0.5,0.00,0.0,0.3,17.80
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,40.761909,1,6.5,0.0,0.5,1.00,0.0,0.3,8.30
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,40.745438,1,8.0,0.0,0.5,2.20,0.0,0.3,11.00
3,2,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,N,-73.971527,40.760330,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
4,1,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,N,-73.982162,40.758999,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30


In [6]:
# what was the mean distance that taxis traveled with 1 passenger?

(
    df
    .loc[lambda df_: df_['passenger_count'] == 1,    # row selector
         'trip_distance']   # column selector
    .mean()
)

np.float64(3.0923380047176354)

In [8]:
# what was the mean distance that taxis traveled with 2 passengers?

(
    df
    .loc[lambda df_: df_['passenger_count'] == 2,    # row selector
         'trip_distance']   # column selector
    .mean()
)

np.float64(3.3843869002284848)

In [9]:
# what about 3 passengers?

(
    df
    .loc[lambda df_: df_['passenger_count'] == 3,    # row selector
         'trip_distance']   # column selector
    .mean()
)

np.float64(3.3423891625615765)

In [11]:
# Let's find out the mean trip_distance for every distinct value of passenger_count



# Grouping

- We use the `groupby` method
- Inside of the parentheses, we pass the name of the categorical column we want to group on
- After the parentheses, we have square brackets with the name of the numeric column we want to calculate on
- After that, we invoke the aggregation method we want

In [12]:
#            categorical        numeric          aggregation function
df.groupby('passenger_count')['trip_distance'].mean()

passenger_count
0    4.600000
1    3.092338
2    3.384387
3    3.342389
4    3.628901
5    3.182712
6    3.170976
Name: trip_distance, dtype: float64

In [15]:
# can I put a numeric column instead of the categorical?
# yes, but you probably don't want to 

# here, we'll ask for the mean trip_distance for every 
# distinct amount that people paid for their taxi
df.groupby('total_amount')['trip_distance'].mean()  

total_amount
-7.80       0.89
-7.30       0.93
-3.30       0.00
 0.30       0.50
 2.30       0.00
           ...  
 162.39    32.10
 194.30    29.30
 210.14    37.20
 250.30     0.00
 252.35    31.90
Name: trip_distance, Length: 916, dtype: float64

# Aggregation methods

- `count`  (returns an integer, the number of non-`NA`/`NaN` values 
- `sum`
- `mean`
- `std`
- `min`
- `max`
- `median`
- `quantile`
- `idxmin`  (returns the index of the lowest value)
- `idxmax`  (returns the index of the highest value)


In [18]:
df['passenger_count'].value_counts()

passenger_count
1    7207
2    1313
5     520
3     406
6     369
4     182
0       2
Name: count, dtype: int64

# When would we use this?

- Find mean sales per region
- Find max sales per salespeople (find the best salesperson!)
- Find std in temperature
- Find the class average for each class in a school

# Exercise: Taxi grouping

You can get this file from GitHub, or you can get it as part of the zipfile here: https://files.lerner.co.il/data-science-exercise-files.zip

1. Find the mean `total_amount` for each value of `passenger_count`. Should a driver prefer to pick up more people? Does it make a big difference?
2. Calculate the percentage tip, using `tip_amount` and `total_amount`. Do you see a difference in the mean tip percentage between vendors of different taxi computers? (Check VendorID)
3. Count how many trips there were for each payment type.

In [19]:

df['payment_type']

0       2
1       1
2       1
3       1
4       2
       ..
9994    2
9995    1
9996    2
9997    2
9998    1
Name: payment_type, Length: 9999, dtype: int64

In [22]:
df.groupby('passenger_count')['passenger_count'].count()

passenger_count
0       2
1    7207
2    1313
3     406
4     182
5     520
6     369
Name: passenger_count, dtype: int64

In [23]:
# 1. Find the mean `total_amount` for each value of `passenger_count`. Should a driver prefer to pick up more people? Does it make a big difference?

df.groupby('passenger_count')['total_amount'].mean()

passenger_count
0    25.570000
1    17.368569
2    18.406306
3    17.994704
4    18.881648
5    17.211269
6    17.401355
Name: total_amount, dtype: float64

In [25]:
# 2. Calculate the percentage tip, using `tip_amount` and `total_amount`. Do you see a difference in the mean 
# tip percentage between vendors of different taxi computers? (Check VendorID)

(
    df
    .assign(tip_percentage = lambda df_: df_['tip_amount'] / df_['total_amount'])
    .groupby('passenger_count')['tip_percentage'].mean()
)


passenger_count
0    0.183127
1    0.092880
2    0.088309
3    0.087368
4    0.077067
5    0.094349
6    0.086075
Name: tip_percentage, dtype: float64

In [27]:
# let's remove all non-tippers, and see how much, on average, people tip
# if they tip at all

(
    df
    .loc[lambda df_: df_['tip_amount'] > 0]   # keep only the rows where there was a tip
    .assign(tip_percentage = lambda df_: df_['tip_amount'] / df_['total_amount'])
    .groupby('passenger_count')['tip_percentage'].mean()
)


passenger_count
0    0.183127
1    0.159227
2    0.160374
3    0.161970
4    0.168990
5    0.162455
6    0.162049
Name: tip_percentage, dtype: float64

In [29]:
(
    df
    .loc[lambda df_: df_['tip_amount'] > 0]   # keep only the rows where there was a tip
    .assign(tip_percentage = lambda df_: df_['tip_amount'] / df_['total_amount'])
    .groupby('VendorID')['tip_percentage'].mean()
)

VendorID
1    0.158171
2    0.161535
Name: tip_percentage, dtype: float64

In [30]:
# 3. Count how many trips there were for each payment type.

df.groupby('payment_type')['total_amount'].count()

payment_type
1    5968
2    3991
3      34
4       6
Name: total_amount, dtype: int64

# How can our groupings get more complex?

We'll start by calculating on more than one column. We can do this by putting a list of strings in the 
square brackets, rather than a single string.

In [31]:
df.groupby('payment_type')[['total_amount', 'trip_distance']].mean()

,total_amount,trip_distance
payment_type,,
1,19.968432,3.438175
2,13.975392,2.745352
3,14.708235,2.765588
4,9.950000,2.031667


We can also get more complex by having more than one categorical column.  Here, the combination of columns
will serve as our index. We'll get one result for each combination.  The resulting series will contain
a multi-index in which the columns appear in the order specified.

In [32]:
df.groupby(['payment_type', 'VendorID'])['total_amount'].mean()

payment_type  VendorID
1             1           19.902497
              2           20.029719
2             1           13.500875
              2           14.415754
3             1           15.958750
              2           -5.300000
4             1           13.500000
              2           -7.800000
Name: total_amount, dtype: float64

# Exercise: Olympic grouping

1. Use the `Olympic` file in the same directory as Jupyter, and turn it into a Pandas data frame.
2. Find the mean height of athletes per `Team`.
3. Find the mean height and weight of athletes per `Team`.
4. Find the mean height of athletes per combination of `Team` and `Year`.


In [33]:
filename = 'olympic_athlete_events.zip'

df = pd.read_csv(filename)

In [34]:
df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [37]:
# mean height of athletes per team

(
    df
    .loc[lambda df_: df_['Year'] > 1960]
    .groupby('Team')['Height'].mean()
    .head(20)
)

Team
Acipactli              174.666667
Afghanistan            170.552632
Akatonbo               182.000000
Alain IV               176.000000
Albania                173.000000
Aldebaran              174.500000
Algeria                174.702869
Ali-Baba IX            175.000000
Almaz                  175.500000
American Samoa         175.666667
Andorra                173.703704
Andromeda              179.000000
Angola                 178.204082
Antigua and Barbuda    175.121739
Aphrodite              182.000000
Argentina              176.879098
Argentina-1            185.125000
Argentina-2            186.125000
Argeste                177.666667
Armenia                171.937799
Name: Height, dtype: float64